# 装饰器

装饰器很好用，而且本身并不复杂。

装饰器是一种用于修改函数或方法行为的高阶函数，它们允许你在不修改函数本身代码的情况下，添加额外的功能。装饰器通常用于日志记录、访问控制、性能测量等场景。

所谓的高阶函数满足以下至少一个条件：

1. 接受一个或多个函数作为参数。
2. 返回一个函数作为结果。

以下为装饰器的简单例子：

In [17]:
import time

def my_decorator(func):
    def wrapper():
        start = time.time()
        func()
        stop = time.time()
        print(f"Execution time:  {(stop-start) * 1000:.3f} ms")
    return wrapper

@my_decorator
def say_hello():
    print("Hello!")

say_hello()

Hello!
Execution time:  0.056 ms


上面装饰器的使用等价于下面的代码：

In [1]:
import time

def my_decorator(func):
    def wrapper():
        start = time.time()
        func()
        stop = time.time()
        print(f"Execution time:  {(stop-start) * 1000:.3f} ms")
    return wrapper

def say_hello():
    print("Hello!")

fn = my_decorator(say_hello)
fn()
print(say_hello.__name__)

Hello!
Execution time:  0.080 ms
say_hello


## 使用 wraps() 装饰器保持函数元数据

从前面的例子可以看出，`say_hello` 函数经 `my_decorator` 装饰器包裹后，实际上调用的是 `wrapper` 函数。因此如果用户获取 `say_hello` 的名字的时候，会发现得到的名字是 `wrapper`。除了这个问题外，其它函数的额外属性也会出现这个问题，例如文档属性等等。

为了解决上述问题，需要使用 `@wraps` 装饰函数：

In [2]:
import time
from functools import wraps

def my_decorator(func):
    @wraps(func)
    def wrapper():
        start = time.time()
        func()
        stop = time.time()
        print(f"Execution time:  {(stop-start) * 1000:.3f} ms")
    return wrapper

@my_decorator
def say_hello():
    print("Hello!")

print(say_hello.__name__)

say_hello


## 原函数带参数

如果原函数有参数，则装饰器的 `wrapper` 函数需要使用任意参数列表：

In [ ]:
import time
from functools import wraps

def my_decorator(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        func(*args, **kwargs)
        stop = time.time()
        print(f"Execution time:  {(stop-start) * 1000:.3f} ms")
    return wrapper

@my_decorator
def say_hello(name):
    print(f"Hello {name}!")

say_hello("John")

Hello John!
Execution time:  0.084 ms


## 带参数的装饰器

带参数的装饰器允许我们在装饰器中传递参数，从而使装饰器更加灵活。下面是一个带参数的装饰器的示例：

In [4]:
import time
from functools import wraps

def my_decorator_with_args(arg1, arg2):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            start = time.time()
            func(*args, **kwargs)
            stop = time.time()
            print(f"Execution time:  {(stop-start) * 1000:.3f} ms")
            print(f"Decorator arguments: {arg1}, {arg2}")
        return wrapper
    return decorator

@my_decorator_with_args("arg1_value", "arg2_value")
def say_hello(name):
    print(f"Hello {name}!")

say_hello("John")

Hello John!
Execution time:  0.080 ms
Decorator arguments: arg1_value, arg2_value


为了增加对参数的支持，装饰器再原本的两层嵌套函数上又加了一层，上面例子中装饰器应用代码等价于：

In [3]:
import time
from functools import wraps

def my_decorator_with_args(arg1, arg2):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            start = time.time()
            func(*args, **kwargs)
            stop = time.time()
            print(f"Execution time:  {(stop-start) * 1000:.3f} ms")
            print(f"Decorator arguments: {arg1}, {arg2}")
        return wrapper
    return decorator

def say_hello(name):
    print(f"Hello {name}!")

_decorator = my_decorator_with_args("arg1_value", "arg2_value")
fn = _decorator(say_hello)
fn("John")

Hello John!
Execution time:  0.081 ms
Decorator arguments: arg1_value, arg2_value


### 实现可选参数装饰器

从前面的例子可以参数，接受参数与不接受参数的装饰器代码有很大的区别，因为前者会比后者多一层函数嵌套。当实现了一个接受参数的装饰器后，即便所有参数都是有默认值的可选参数，也必须在使用装饰器时加上括号，但这无疑增加了装饰器的使用成本：

In [6]:
import time
from functools import wraps

def my_decorator_with_args(arg1='foo', arg2="bar"):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            start = time.time()
            func(*args, **kwargs)
            stop = time.time()
            print(f"Execution time:  {(stop-start) * 1000:.3f} ms")
            print(f"Decorator arguments: {arg1}, {arg2}")
        return wrapper
    return decorator

@my_decorator_with_args()
def say_hello(name):
    print(f"Hello {name}!")

say_hello("John")

Hello John!
Execution time:  0.068 ms
Decorator arguments: foo, bar


我们可以对装饰器进行改造，让在使用参数默认值的时候不需要添加括号：

In [11]:
import time
from functools import wraps

def my_decorator_with_args(func=None, *, arg1='foo', arg2="bar"):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            start = time.time()
            func(*args, **kwargs)
            stop = time.time()
            print(f"Execution time:  {(stop-start) * 1000:.3f} ms")
            print(f"Decorator arguments: {arg1}, {arg2}")
        return wrapper

    if func is None:
        return decorator
    else:
        return decorator(func)

@my_decorator_with_args
def say_hello(name):
    print(f"Hello {name}!")

say_hello("John")

@my_decorator_with_args(arg1='foo1', arg2='foo2')
def say_hello_1(name):
    print(f"Hello {name}!")

say_hello_1("Ryan")

Hello John!
Execution time:  0.634 ms
Decorator arguments: foo, bar
Hello Ryan!
Execution time:  0.008 ms
Decorator arguments: foo1, foo2
